<a href="https://colab.research.google.com/github/CallumPaton/analysing-most-competitve-f1-season/blob/main/F1_data_analysis_changes_to_championship_standings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive

In [ ]:
# Mount the google drive containing data taken from kaggle (https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020)
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/F1-data-analysis'
driver_standings = pd.read_csv(f'{file_path}/driver_standings.csv')
races = pd.read_csv(f'{file_path}/races.csv')

In [ ]:
drivers = pd.read_csv(f'{file_path}/drivers.csv')

In [ ]:
def FindDriver(driverId):
  print(drivers[drivers['driverId']==driverId]['driverRef'].to_string(index=False))

In [ ]:
races.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [ ]:
# Adding the season ID to the driver standings data
driver_standings = driver_standings.merge(races[['raceId','year','round']], on='raceId')

In [ ]:
driver_standings.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins,year,round
0,1,18,1,10.0,1,1,1,2008,1
1,2,18,2,8.0,2,2,0,2008,1
2,3,18,3,6.0,3,3,0,2008,1
3,4,18,4,5.0,4,4,0,2008,1
4,5,18,5,4.0,5,5,0,2008,1


In [ ]:
# Work out how much the order shuffled around between races
import numpy as np

def map_to_zero_or_one(value):
    if value > 0:
        return 1
    else:
        return 0

def changes_to_drivers_standings_in_top_n(n):
  # Create a new dataframe to store the results
  result_df = pd.DataFrame(columns=['driverId', 'year', 'total_no._order_changes'])

  # group the dataframe by year
  grouped_by_season = driver_standings.groupby('year')

  dfs_to_concat =[]

  for year, season_data in grouped_by_season:
    # make it such that the position is capped at n.
    season_data['position'] = np.where(season_data['position'] > n, n+1, season_data['position'])

    season_data['absolute_position_change'] = season_data.groupby('driverId')['position'].diff().abs()
    season_data = season_data[['raceId','driverId','position','year','absolute_position_change','round']]
    season_data['absolute_position_change'] = np.where(season_data['position'] > n, 0.0, season_data['absolute_position_change'])

    # map a change in championship order to a binary value
    season_data['absolute_position_change'] = season_data['absolute_position_change'].apply(map_to_zero_or_one)
    # Calculate the total absolute position change for each driver in the season
    driver_summary = season_data.groupby('driverId')['absolute_position_change'].sum().reset_index()
    number_of_races = season_data['raceId'].nunique()

    # Add the 'year' column to the result
    driver_summary['year'] = year
    driver_summary['number_of_races'] = number_of_races
    # Append the results to the result_df
    dfs_to_concat.append(driver_summary)

  # Reset the index of the result_df
  result_df= pd.concat(dfs_to_concat, ignore_index = True)

  # Remove rows where 'year' is less than 1980 or equal to 2023
  filtered_df = result_df[(result_df['year'] >= 1990) & (result_df['year'] < 2023)]

  # Group by 'year' and calculate total position changes and total races
  # Group by 'year' and calculate the sum of 'absolute_position_change' and 'number_of_races'
  grouped = filtered_df.groupby('year').agg({'absolute_position_change': 'sum', 'number_of_races': 'max'})

  # Calculate the average of 'absolute_position_change' divided by 'number_of_races'
  grouped[f'avg. no. order changes per race'] = (grouped['absolute_position_change'] / grouped['number_of_races'])

  grouped['n'] = n
  # Reset the index to have 'year' as a regular column
  grouped.reset_index(inplace=True)

  return grouped[['year','n',f'avg. no. order changes per race']]

In [ ]:
changes_to_drivers_standings_in_top_n(1)

<ipython-input-74-cff6748751ca>:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-74-cff6748751ca>:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-74-cff6748751ca>:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-74-cff6

,year,n,avg. no. order changes per race
0,1990,1,0.125000
1,1991,1,0.000000
2,1992,1,0.000000
3,1993,1,0.250000
4,1994,1,0.000000
5,1995,1,0.117647
6,1996,1,0.000000
7,1997,1,0.411765
8,1998,1,0.000000
9,1999,1,0.375000


In [ ]:
FindDriver(driverId=22)

barrichello


In [ ]:
import matplotlib.pyplot as plt

df_list = []
for n in range(1,21):
  df = changes_to_drivers_standings_in_top_n(n)
  df_list.append(df)

idk = pd.concat(df_list, ignore_index = True)

Streaming output truncated to the last 5000 lines.
<ipython-input-74-cff6748751ca>:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-74-cff6748751ca>:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-74-cff6748751ca>:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    html.Label('Select years to display:',style={'fontSize':40}),
    dcc.Checklist(
        id='year-checklist',
        options=[{'label': str(year), 'value': year} for year in idk['year'].unique()],
        value=[df['year'].min()],
        style={'fontsize':18}# Default to the minimum year
    ),
    dcc.Graph(id='line-plot',
              style={'width':'80%','height':'500px'}),
])

# Define callback to update the plot based on selected years
@app.callback(
    Output('line-plot', 'figure'),
    [Input('year-checklist', 'value')]
)
def update_line_plot(selected_years):
    filtered_df = idk[idk['year'].isin(selected_years)]
    traces = []
    for year, group_data in filtered_df.groupby('year'):
        trace = go.Scatter(
            x=group_data['n'],
            y=group_data['avg. no. order changes per race'],
            mode='lines+markers',
            name=f'Year {year}',
        )
        traces.append(trace)

    return {
        'data': traces,
        'layout': go.Layout(
            xaxis={'title': 'Top n Positions','titlefont':{'size':22}},
            yaxis={'title': 'Avg. No. Standings Changes per Race','titlefont':{'size':22}},
            legend={'font':{'size':18}},
            showlegend=True,
        )
    }

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

In [ ]:
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 10.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.3.7
    Uninstalling Werkzeug-2.3.7:
      Successfully uninstalled Werkzeug-2.3.7
